### Naive Approach

In [1]:
# !pip install sent2vec

In [2]:
from scipy import spatial
from datasets import load_dataset
from sent2vec.vectorizer import Vectorizer
import numpy as np
from numpy import dot
from numpy.linalg import norm
from annoy import AnnoyIndex

df = load_dataset('paws', 'labeled_final')

train = df['train'].to_pandas()
validation = df['validation'].to_pandas()
test = df['test'].to_pandas()
# train.head()
test.head()

,id,sentence1,sentence2,label
0,1,This was a series of nested angular standards ...,"This was a series of nested polar scales , so ...",0
1,2,His father emigrated to Missouri in 1868 but r...,"His father emigrated to America in 1868 , but ...",0
2,3,"In January 2011 , the Deputy Secretary General...","In January 2011 , FIBA Asia deputy secretary g...",1
3,4,"Steiner argued that , in the right circumstanc...",Steiner held that the spiritual world can be r...,0
4,5,"Luciano Williames Dias ( born July 25 , 1970 )...",Luciano Williames Dias ( born 25 July 1970 ) i...,0


In [3]:
# train_sentence1 = train['sentence1'][0:1500].tolist()
# train_sentence2 = train['sentence2'][0:1500].tolist()
# train_id = train['id'].tolist()

test_sentence1 = train['sentence1'][0:1500].tolist()
test_sentence2 = train['sentence2'][0:1500].tolist()
test_id = test['id'].tolist()


# train_sentence1
vectorizer = Vectorizer()
# vectorizer.run(train_sentence1)
# vectors_TS1_bert = vectorizer.vectors

# vectorizer.run(train_sentence2)
# vectors_TS2_bert = vectorizer.vectors

vectorizer.run(test_sentence1)
vectors_TST1_bert = vectorizer.vectors

vectorizer.run(test_sentence2)
vectors_TST2_bert = vectorizer.vectors

Initializing Bert distilbert-base-uncased
Vectorization done on cpu


We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


In [7]:
from sklearn.metrics.pairwise import cosine_similarity

In [8]:
count = sum(train['label'] == 1)
total_pairs = (max(train['id']))
threshold = count/total_pairs
threshold

0.4418736462824639

In [9]:
paraphrasing = []
# for i in vectors_TS1_bert, vectors_TS2_bert,train['label'][0:1500]:
#     similarity_score = cosine(vectors_TS1_bert[i], vectors_TS2_bert[i]) 
#     if similarity_score >= threshold:
#         paraphrasing.append(1)
#     else:
#         paraphrasing.append(0)

for i in range(len(vectors_TST1_bert)):
    similarity_score = cosine_similarity([vectors_TST1_bert[i]], [vectors_TST2_bert[i]])[0][0]
    # print(similarity_score)
    if similarity_score >= threshold:
        paraphrasing.append(1)
    else:
        paraphrasing.append(0)

In [16]:
len(vectors_TST1_bert)

3000

In [13]:
paraphrasing.count(1)

3000

Since all the words are paraphrases so they have similarity socre of 1 so to differentiate that we can use **parts of speech taging** and **check if all the parts of speech match or not**.

Since all the sentences are paraphrases $\implies$ they are supposed to be same
for example:
sentence-1: This was a series of nested **angular standards** , so that measurements in azimuth and elevation could be done directly in polar coordinates relative to the ecliptic .
sentence-2: This was a series of nested **polar scales** , so that measurements in azimuth and elevation could be performed directly in angular coordinates relative to the ecliptic

In both of these sentences only one word is changed i.e `angular standards` &#8594; `polar scales`.
so to find similarity when these types of changes are done we can't use sentence vectorization and then checking similarity using euclidian/cosine similarity but rather we should move for **supervised learning** for accurate prediction.

# Using Parts Of Speech Tagging for Sentence similarity

## Accuracy of the model is:55.800000000000004

In [18]:
from scipy import spatial
from datasets import load_dataset
from sent2vec.vectorizer import Vectorizer
import numpy as np
from numpy import dot
from numpy.linalg import norm
from annoy import AnnoyIndex


def cosine(v1, v2):
    if norm(v1) > 0 and norm(v2) > 0:
        return dot(v1, v2) / (norm(v1) * norm(v2))
    else:
        return 0.0

df = load_dataset('paws', 'labeled_final')

train = df['train'].to_pandas()
validation = df['validation'].to_pandas()
test = df['test'].to_pandas()
# train.head()
test.head()

,id,sentence1,sentence2,label
0,1,This was a series of nested angular standards ...,"This was a series of nested polar scales , so ...",0
1,2,His father emigrated to Missouri in 1868 but r...,"His father emigrated to America in 1868 , but ...",0
2,3,"In January 2011 , the Deputy Secretary General...","In January 2011 , FIBA Asia deputy secretary g...",1
3,4,"Steiner argued that , in the right circumstanc...",Steiner held that the spiritual world can be r...,0
4,5,"Luciano Williames Dias ( born July 25 , 1970 )...",Luciano Williames Dias ( born 25 July 1970 ) i...,0


In [19]:
count = sum(train['label'] == 1)
total_pairs = (max(train['id']))
threshold = count/total_pairs
threshold

0.4418736462824639

In [20]:
import spacy
nlp = spacy.load('en_core_web_lg')

In [24]:
correct_score = []
# for i in range(2):
for i in range((max(test['id']))):
    s1 = nlp(test['sentence1'][i])
    s1_verbs = " ".join([token.lemma_ for token in s1 if token.pos_ == "VERB"])
    s1_adj = " ".join([token.lemma_ for token in s1 if token.pos_ == "ADJ"])
    s1_nouns = " ".join([token.lemma_ for token in s1 if token.pos_ == "NOUN"])
    
    s2 = nlp(test['sentence2'][i])
    s2_verbs = " ".join([token.lemma_ for token in s2 if token.pos_ == "VERB"])
    s2_adj = " ".join([token.lemma_ for token in s2 if token.pos_ == "ADJ"])
    s2_nouns = " ".join([token.lemma_ for token in s2 if token.pos_ == "NOUN"])
    
    similarity_score = s1.similarity(s2)
    score_verb = nlp(s1_verbs).similarity(nlp(s2_verbs))
    score_adj = nlp(s1_adj).similarity(nlp(s2_adj))
    score_noun = nlp(s1_nouns).similarity(nlp(s2_nouns))
    # print(f"sentence1: {s1}\n")
    # print(f"nouns:{s1_nouns}\t adj:{s1_adj}\t verb:{s1_verbs}\n")
    # print(f"sentence2: {s2}\n")
    # print(f"nouns:{s2_nouns}\t adj:{s2_adj}\t verb:{s2_verbs}\n")
    # print(f"sim-score:{similarity_score}, adj_score:{score_adj}, noun-sc:{score_noun}, verb_score:{score_verb}\n")
    # print("-------------------------")
    if similarity_score == threshold and score_noun == 1.0 and score_verb == 1.0 and score_adj == 1.0:
        correct_score.append(1)
    else:
        correct_score.append(0)

/var/folders/02/mzwwwjm533sgdkcfymq6ntdh0000gn/T/ipykernel_97050/1238688201.py:16: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  score_adj = nlp(s1_adj).similarity(nlp(s2_adj))
/var/folders/02/mzwwwjm533sgdkcfymq6ntdh0000gn/T/ipykernel_97050/1238688201.py:15: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  score_verb = nlp(s1_verbs).similarity(nlp(s2_verbs))
/var/folders/02/mzwwwjm533sgdkcfymq6ntdh0000gn/T/ipykernel_97050/1238688201.py:17: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  score_noun = nlp(s1_nouns).similarity(nlp(s2_nouns))


In [25]:
# correct_score

In [26]:
actual_scores = np.array(test['label'])
correct_score_arr = np.array(correct_score)
predicted_scores = np.sum(correct_score_arr == actual_scores)
print(predicted_scores)
accuracy = predicted_scores / len(actual_scores)
print(f"Accuracy of the model is:{accuracy*100}")

4464
Accuracy of the model is:55.800000000000004
